# Detroit Blight
A study notebook

Overall plan <br>
1) Load  incident data from the various files<br>
2) Define 'houses' as  geographic entities, for example a 10 x 10 meter squares. <br>
3) Allocate incidents to houses and count the number of incidens for the house. <br>
4) Allocate known demolition permits to houses. Houses with a demolition permit are regarded as blighted. <br>
5) Train a model to predict which houses are marked for demolition (=are blighted) <br>
 a) simple model by count of incidents <br>
 b) intermediate model by count of incidents per category

## Data
Downloaded these files. Using the local files in the code.
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-blight-violations.csv
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-311.csv
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-crime.csv
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-demolition-permits.tsv





## Prepare for data manipulation


In [1]:
    library(data.table)
    library(pbapply)
    library(stringr)
    library(tree)
    library(ROCR)

setDirs <- function(){
    basedir <-
        "C:/Users/setup/Documents/coursera/uw/datascience/capstone"
    assign("datadir",(paste0(basedir, "/data")),.GlobalEnv)
    assign("codedir",(paste0(basedir, "/code")),.GlobalEnv)
    setwd(datadir)    
}

loadDf <- function(fileName) {
    setDirs()
    fullName <- paste0(datadir, "/", fileName)
    # tryCatch(
        df <- read.csv(fullName, stringsAsFactors = FALSE)
#         error = function(e) "File not found!"
#     )
    return(df)
}
loadDt <- function(fileName){
    d <- loadDf(fileName)
    d <- data.table(d)
    return(d)
}

Warning message:
: package 'data.table' was built under R version 3.2.5Warning message:
: package 'pbapply' was built under R version 3.2.5Warning message:
: package 'tree' was built under R version 3.2.5Warning message:
: package 'ROCR' was built under R version 3.2.5Loading required package: gplots
Warning message:
: package 'gplots' was built under R version 3.2.5
Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess



Blight violations
First load and format blight violation data

In [3]:
getBViol <- function(){
    bv <- loadDt("detroit-blight-violations.csv")
    return(bv)
}

formatBv <- function(bv) {
    library(pbapply)
    extractCoord <- function(bvsRow) {
        # "XAddress" is like ..., 2566 GRAND BLVD\nDetroit, MI\n(42.36318237000006, -83.09167672099994), ...
        # rowstr <- unlist(strsplit(toString(bvsRow),split = "\n"))
        vcoord <- unlist(strsplit(toString(bvsRow["ViolationAddress"]),split = "\n"))[3]
        # Now we have something like (42.32544917300004, -83.06413908999997)
        # vcoord <- gsub("[()]","",vcoord)
        # vcoord <- unlist(strsplit(vcoord,split = ","))
        mcoord <- unlist(strsplit(toString(bvsRow["MailingAddress"]),split = "\n"))[3]
        coord <- paste(vcoord,",", mcoord)
        coord <- gsub("[()]","",coord)
        coord <- unlist(strsplit(coord,split = ","))
        # coord <- vcoord
        return(coord)
    }
    
    coord <- t(pbapply(bv,1,extractCoord))
    # Add Violation and Mailing latitude and longitude as columns
    splitCoord <- function(crow){
        c <- c(crow[1],crow[2],crow[3],crow[4])
    }
    coord <- t(pbsapply(coord,splitCoord))
    bv$Lat <- as.numeric(coord[,1])
    bv$Lon <- as.numeric(coord[,2])
    bv$MLat <- as.numeric(coord[,3])
    bv$MLon <- as.numeric(coord[,4])

    # Add a column that indicates if the violation and mail addresses are equal (a proxy for the owner living in the building)
    bv$ViolEqMail <- (bv$Lat == bv$MLat & bv$Lon == bv$MLon)
    # Add a column for engineered violation category (and turn bv into a data.table)
    print("Start categorizing Blight Violations")
    bv <- categorizeBv(bv)
    setkey(bv,Lat,Lon,VCategory)
    bv <- subset(bv, select = c("VCategory", "ViolName", "ViolationCode", "Lat", "Lon", "ViolEqMail"))
    return(bv)
} # formatBv

bv <- getBViol()
bv <- formatBv(bv)
head(bv,2)

Warning message:
In formatBv(bv): NAs introduced by coercion

[1] "Start categorizing Blight Violations"


ERROR: Error in formatBv(bv): could not find function "categorizeBv"


,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,ellip.h,AdminFee,LateFee,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress
1,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,<8b>,$20.00,$150.00,$10.00,$0.00,$1680.00,PAID IN FULL,0,0,"2566 GRAND BLVD Detroit, MI (42.36318237000006, -83.09167672099994)","743 Beaubien Detroit, MI 48226 (42.33373063000005, -83.04181755199994)"
2,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,<8b>,$20.00,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0,0,"19014 ASHTON Detroit, MI (42.429390762000025, -83.22039357799997)","20501 HEYDEN DETROIT, MI 48219 (42.44217763300003, -83.24182717199994)"


In [11]:
getD311 <- function() {
    d311 <- loadDf("detroit-311.csv")
    #u311type <- unique(d311$issue_type,use.names = FALSE)
    d311 <- subset(
        d311,
        select = c(
            issue_type,ticket_closed_date_time,ticket_created_date_time,
            address,lat,lng
        )
    )
    # assign("u311type",u311type,envir = .GlobalEnv)
    n311 <- names(d311)
    n311[1] <- "IncType"
    n311[5] <- "Lat"
    n311[6] <- "Lon"
    names(d311) <- n311
    d311$Lat <- as.numeric(d311$Lat)
    d311$Lon <- as.numeric(d311$Lon)
    d311 <- data.table(d311)
    # Guessing which incident types could be attributed to a house and indicate blight
    d311 <- d311[IncType %in% c("Abandoned Vehicle", "Curbside Solid Waste Issue", "Illegal Dumping / Illegal Dump Sites",
                                "Running Water in a Home or Building", "Trash Issue - Improper placement of refuse container between collections/left at curbside")]
    return(d311)
}

## 2) Build houses
Now that the functions are defined, let's actually build the houses and limit to those within Detroit's boundaries.

In [15]:
buildHouses <- function(precision, bv){
    inc <- data.table(Lat = bv$Lat, Lon = bv$Lon, IncType = bv$VCategory, ViolEqMail = bv$ViolEqMail
                      )
    rm(bv)
    d311 <- getD311()
    d311$ViolEqMail <- "NA"
    d311 <- categorize311(d311)
    d311$IncType <- d311$d311Category
    d311 <- subset(d311, select=c("Lat", "Lon", "IncType", "ViolEqMail"))

    inc <- rbind(inc, d311)
    rm(d311)
# At a closer look, crime incidents are not clearly linked to a building, so it's not used here to determine houses.

    # Remove clear geographic outliers. Coordinates from Detroit bounding box with a margin.
    inc <- inc[Lat %between% c(42.25,42.5) & Lon %between% c(-83.3,-82.9)]
    # Then against Detroit city boundaries
    inc <- overDetroit(inc)
    print(paste('Number of incidents in Detroit total: ', nrow(inc)))
    inc <- data.table(inc)
    inc$Lat <- round(as.numeric(inc$Lat),digits=precision)
    inc$Lon <- round(as.numeric(inc$Lon),digits=precision)
    setkey(inc, Lat, Lon, IncType)
    # The data table operator .N adds the number of incidents in the by group to every member of the group. 
    # We add it as a new column Count, which has the number of incidents per rounded location and type
    inc <- inc[,":=" (Count = .N), by= list(Lat, Lon, IncType)]
    # We need separate columns for each incident type
    inc <- addCountColumnsPerIncType(inc)
    inc <- unique(inc)
    inc <- inc[ , j=list(IncType, ViolEqMail, max(TrashIncCount), max(PermitIncCount), max(MaintenanceIncCount), max(VehicleIncCount), max(WaterIncCount)), by=list(Lat, Lon)]
    colnames(inc) <- c("Lat", "Lon", "IncType", "ViolEqMail", "Trash", "Permit", "Maintenance", "Vehicle", "Water")
    # Now we still need to remove duplicates (incidents with identical coordinates, when rounded to precision)(DT uses key fiels to determine uniqueness)
    setkey(inc, Lat, Lon)
    houses <- unique(inc)
    rm(inc)
    setkey(houses, Lat, Lon)
    # Houses are now sorted by ascending Lat, Lon. Add row number as unique HouseId
    houses <- cbind(row.names(houses),houses)
    cnh <- colnames(houses)
    cnh[1] <- "HouseId"
    colnames(houses) <- cnh
    st <- sum(houses$Trash) + sum(houses$Permit)+ sum(houses$Maintenance) + sum(houses$Vehicle) + sum(houses$Water)  
    print(paste('Total counts in summary table houses: ', st))
    return(houses)
}

overDetroit <- function(dt){
    # dt must be a data table with columsn Lon and Lat with no NAs
    library(sp)
    library(rgdal)
    coordinates(dt) <- ~Lon + Lat
    detroitShapeDir <- paste0(datadir, "/City Boundaries shapefile")
    detroit <- readOGR(dsn=detroitShapeDir, layer = "geo_export_941b9f41-e08a-40db-ae36-d77a7046e1a5")
    proj4string(dt) <- proj4string(detroit)
    dInD <- dt[detroit,] #shorthand for dt[!is.na(over(dt,detroit)),]
    dt <- tryCatch(
        cbind(dInD@coords[,2],dInD@coords[,1], dInD@data), # add Lat, Lon
        error = function(e){
            print("Could not find additional data in dt!")
            return(data.table(Lat=dInD@coords[,2],Lon=dInD@coords[,1]))
        }
    )
    colnames(dt)[1:2] <- c("Lat", "Lon")
    dt <- data.table(dt)
    return(dt)
}

OGR data source with driver: ESRI Shapefile 
Source: "C:/Users/setup/Documents/coursera/uw/datascience/capstone/data/City Boundaries shapefile", layer: "geo_export_941b9f41-e08a-40db-ae36-d77a7046e1a5"
with 1 features
It has 7 fields


# 3) Get demolition permits
Load demolition permit data. Assign permits to previously defined houses. As a result, we have a list of demolition permits with a HouseId, if available. Many of the permits don't belong to any of the houses we've constructed above.
Again, first define functions, then finally use them.


In [ ]:
getDemolition <- function(){
        # if(is.null(dem)){
            dem <- read.csv(file=
                                "C:/Users/setup/Documents/coursera/uw/datascience/capstone/data/detroit-demolition-permits.tsv",
                            sep = "\t", stringsAsFactors = FALSE)
        # }
#         dem <- subset(dem, select=c(PERMIT_ISSUED, CASE_TYPE, BLD_PERMIT_TYPE, site_location))
        return(dem)
}

formatDemolition <- function(dem){
    library(stringr)
    siteLoc <- strsplit(dem$site_location, split = "\n")
    siteLocLen <- unlist(Map(length,siteLoc))
    
    # Get the coordinates
    crd <- str_extract_all(dem$site_location, "[(].+[)]", simplify = TRUE) # we get (42.nnnn, -83.nnn)
    crd <- gsub("[()]","",crd)
    crd <- strsplit(crd,split = ",")
    # strsplit returns a list. Now need to replace missing coordinates with NA before Reducing list to matrix,
    # or else the rows won't match.
    crd <- Map(function(crdlistitem){ 
        if(length(crdlistitem==2)) return(crdlistitem) 
        else return(NA) }, crd)
    crd <- Reduce(rbind,crd)
    
    # Get the address
    extractDemAddr <- function(row){
        l <- length(row)
        if(l<1) return(c(NA,NA))
        else if(l==1) return(c(row[1],NA))
        else return(c(row[1],row[2]))
    }
    add <- Map(extractDemAddr,siteLoc)
    add <- t(as.data.frame(x=add, stringsAsFactors=FALSE ))
    dnam <- c("Lat", "Lon", "Street", "City", names(dem))
    dem <- cbind(as.numeric(crd[,1]), as.numeric(crd[,2]), add[,1], add[,2], dem)
    names(dem) <- dnam
    dem$site_location <- gsub("\n"," ",dem$site_location)
    dem$owner_location <- gsub("\n"," ",dem$owner_location)
    dem$contractor_location <- gsub("\n"," ",dem$contractor_location)
    dem$Street <- as.character(dem$Street)
    dem$City <- as.character(dem$City)
    return(dem)
}

prepareDemolition <- function(){
    dem <- getDemolition()
    dem <- formatDemolition(dem)
    dem <- data.table(dem)
    dem <- geoCodeDem(dem)
    # Slice out those in Detroit only!
    dem <- dem[!(is.na(Lat))] #in case there still are some
    dem <- overDetroit(dem)
    dem$Lat <- round(dem$Lat, digits = 4)
    dem$Lon <- round(dem$Lon, digits = 4)
    return(dem)
}

geoCodeDem <- function(dem){
    # Some 900 entries have NA for coordinates. We'll get them from file, if available, or geocode them with ggmap::geocode    
    demToGeocode <- dem[is.na(Lat)] #I've tested that identical(is.na(dem$Lat), is.na(dem$Lon)) 
    if(nrow(demToGeocode) < 1){
        return()
    }
    geocoded <- tryCatch(
        loadDf("geocoded2.csv"),
        error = function(e){
            dem <- dem[!(is.na(Lat))]
            demToGeocode <- get0("demToGeocode")
            addresses <- paste0(demToGeocode$Street, " ",gsub(",", "",demToGeocode$City))
                        # TESTING: First try geocoding with a small sample
            addresses <- head(addresses, 10)
            # Start the geocoding process - address by address. geocode() function takes care of query speed limit.
            library(ggmap)
            geocoded <- data.frame()
            for (ii in (1:length(addresses))){
                if (ii %% 50 == 0){
                    print(paste("Working on index", ii, "of", length(addresses)))    
                }
                #query the google geocoder - this will pause here if we are over the limit.
                result = geoCodeAddress(addresses[ii]) 
                # print(result$status)     
                result$index <- ii
                #append the answer to the results file.
                geocoded <- rbind(geocoded, result)
                #save temporary results as we are going along
                # saveRDS(geocoded, tempfilename)
            }
            # Export the geocoded results for scrutiny
            assign("geocoded",geocoded,.GlobalEnv)
            writeData(data = geocoded, filename="geocoded2.csv")
            return(geocoded)
        }
    )
    # Finally round and add the geocoded entries to dem
    demToGeocode$Lat <- geocoded$Lat
    demToGeocode$Lon <- geocoded$Lon
    dem <- rbind(dem,demToGeocode)
}

geoCodeAddress <- function(address){
    geo_reply = geocode(address, output='all', messaging=TRUE, override_limit=TRUE)
    #now extract the bits that we need from the returned list
    answer <- data.frame(Lat=NA, Lon=NA, accuracy=NA, formatted_address=NA, address_type=NA, status=NA)
    answer$status <- geo_reply$status
    #if we are over the query limit - want to pause for an hour
    while(geo_reply$status == "OVER_QUERY_LIMIT"){
        print("OVER QUERY LIMIT - Pausing for 1 hour at:") 
        time <- Sys.time()
        print(as.character(time))
        Sys.sleep(60*60)
        geo_reply = geocode(address, output='all', messaging=TRUE, override_limit=TRUE)
        answer$status <- geo_reply$status
    }
    if (geo_reply$status != "OK"){
        return(answer)
    }   
    #else, extract what we need from the Google server reply into a dataframe:
    answer$Lat <- geo_reply$results[[1]]$geometry$location$lat
    answer$Lon <- geo_reply$results[[1]]$geometry$location$lng   
    if (length(geo_reply$results[[1]]$types) > 0){
        answer$accuracy <- geo_reply$results[[1]]$types[[1]]
    }
    answer$address_type <- paste(geo_reply$results[[1]]$types, collapse=',')
    answer$formatted_address <- geo_reply$results[[1]]$formatted_address
    return(answer)
}


# Time to put it all together!
Get and format the demolition permits

In [ ]:

dem <- prepareDemolition()
demolitionHouses <- function(dem, houses){
    # Link demolition permits to houses. Both must be data.table, with Lat, Lon rounded to 4
    #   Use a simple inner join of dt!
    # dem: Lat, Lon, Street...  house: HouseId, Count, Lat, Lon.
    setkey(dem, Lat, Lon)
    setkey(houses, Lat, Lon)
    dh <- houses[dem, nomatch=0]
    # Multiple demolition permits have been sent to the same houses, so we must dedup dh
    dh <- unique(dh)
    return(dh)
}

# Assign them to houses
dh <- demolitionHouses(dem,houses)

### Shortcut
Using the file I've saved from my dataframes, load dh

In [ ]:
dh <- read.csv("houses_with_demolition.csv",stringsAsFactors = FALSE)
# Checked in Carto that only 1 of these may fall a few meters outside Detroit --> ignore error
dh <- data.table(dh)

## 4) Prepare demolition permits for training and testing
Many demolition permits are not associated to any of our houses. In other words there are no incidents linked to them. Let's disregard them.

In [ ]:
gh <- dh[!is.na(house)]
sgh <- subset(gh,select=c("house","Lat","Lon","PERMIT_ISSUED","LEGAL_USE","PARCEL_SIZE","STORIES"))
rm(dh,gh)
nrow(sgh)

There should be 3573 demolition permits assigned to houses (sgh).
Check for duplicates ie. multiple permits to the same house:

In [ ]:
> sum(duplicated(sgh$house))
[1] 1090
> sum(duplicated(sgh$house, sgh$Lat, sgh$Lon))
[1] 1090

Finally remove duplicates and prepare the train and test sets.

In [ ]:
sgh <- sgh[!duplicated(sgh$house),] 
n <- nrow(sgh)
train <- sample(1:n, size=round(0.7*n),replace=FALSE)
demtrain <- sgh[train,]
demtest <- sgh[-train,]
# Some crude validity tests
sum(duplicated(sgh$house))
max(sgh$Lat)
max(sgh$Lon)

bHouses <- data.table(HouseId = sgh$house, ToDemolition = TRUE)
nbHouses <- houses[!(HouseId %in% sgh$house)]
nbHouses <- data.table(HouseId = as.integer(nbHouses$HouseId), ToDemolition = FALSE)
nbHouses <- nbHouses[sample(1:nrow(nbHouses), size=nrow(bHouses), replace=FALSE),]
# as result, we have nrow(bHouses) = nrow(nbHouses) = 2483

ttHouses <- rbind(bHouses, nbHouses)
ttHouses[order(ttHouses$HouseId),]
# Ordering by HouseId should distribute the blighted/nonblighted houses fairly randomly, 
# as the house numbers were generated with no reference to blight


# 5) Number of incidents per house in training/test set

In [ ]:
bvc <- loadDf("bvCounts.csv") #A result of previous aggregation of incidents per Lat,Lon

Alternatively, here's how the object in bvCounts was built:

In [1]:
bv <- loadDf("Detroitbviolfromcartocsv.csv")
bv <- subset(bv, select=c("lat", "lon"))
bv$lat <- round(bv$lat,digits=4)
bv$lon <- round(bv$lon,digits=4)
bv <- data.table(bv)
bv <- bv[, ":=" (Count = .N), by= list(lat,lon)]
# The data table operator .N adds the number of incidents in the by group to every member of the group.
# It did not remove duplicates (aggregate to one member per group)
bv <- unique(bv)
names(bv)=c("Lat", "Lon", "Count")
setkey(bv,Lat,Lon)

train <- loadDf("trainSet.csv")
train <- data.table(train)
setkey(train,Lat,Lon)
# Join the data tables
bt <- bv[train]
names(bt)
# [1] "Lat"          "Lon"          "Count"        "HouseId"      "ToDemolition"


ERROR: Error in eval(expr, envir, enclos): could not find function "loadDf"


ERROR: Error in subset(bv, select = c("lat", "lon")): object 'bv' not found


ERROR: Error in eval(expr, envir, enclos): object 'bv' not found


ERROR: Error in eval(expr, envir, enclos): object 'bv' not found


ERROR: Error in eval(expr, envir, enclos): could not find function "data.table"


...

ERROR: Error in eval(expr, envir, enclos): could not find function "loadDf"


ERROR: Error in eval(expr, envir, enclos): could not find function "data.table"


ERROR: Error in eval(expr, envir, enclos): could not find function "setkey"


ERROR: Error in eval(expr, envir, enclos): object 'bv' not found


ERROR: Error in eval(expr, envir, enclos): object 'bt' not found


### 6) Build simple model
Build the model based on solely the number of incidents per case in the training set.

In [ ]:
btree <- tree(ToDemolition ~ Count, data= bt)
cv.tree(btree, , prune.tree,K=5)
plot(tree)
text(tree)

The resulting model is very simple: if the number of incidents >0.5 (in practice if there are incidents) then predict TRUE = to be demolished = blighted.
The precision is not too good.